# Linear Regression + Timeseries
For this mini-lab, you will learn a bit about how to use linear regression to fit _filters_ that you can use to model timeseries. You'll be using linear regression techniques to model fMRI responses to video stimuli (the ones that we've talked about in class) based on the presence of two categories in the videos: people and buildings.



In [3]:
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import r2_score

# Load the fMRI dataset
datafile = np.load('data.npz')

# list all the variables in the file
print(datafile.files)

['ybig_trn', 'X_test', 'X_trn', 'y_trn', 'ybig_test', 'y_test']


In [18]:
# Load all the data
# (you don't need to worry about subtracting the mean later because that's done here)

# the features (X) say whether people (column zero) or buildings (column one) are present in each video clip
X_trn = datafile['X_trn'] # time x features
X_trn -= X_trn.mean(0) # subtract the mean over time
X_test = datafile['X_test'] # time x features
X_test -= X_test.mean(0) # ditto

# y_trn and y_test have the fMRI response of one voxel
y_trn = datafile['y_trn'] # time
y_trn -= y_trn.mean(0)
y_test = datafile['y_test'] # time
y_test -= y_test.mean(0)

# ybig_trn and ybig_test have the response of 10,000 voxels
ybig_trn = datafile['ybig_trn']
ybig_trn -= ybig_trn.mean(0)
ybig_test = datafile['ybig_test']
ybig_test -= ybig_test.mean(0)

In [10]:
# print the size of each array

# Plot the regression target (y) and each of the features
Step 1 in any analysis should _always_ be to **LOOK AT YOUR DATA**. Let's plot it and see what it looks like: what's the range, does it look uniform, etc.

In [11]:
# plot y_trn, X_trn[:,0], and X_trn[:,1] on the same axis. do you see anything?

In [ ]:
# compute the correlation between X_trn[:,0] and y_trn (ditto for X_trn[:,1])

In [12]:
# plot a histogram of y_trn

# The Finite Impulse Response (FIR) Regression Model
One of the key things you should know about fMRI is that it measures the Blood-Oxygen-Level Dependent (BOLD) signal, which tells you about blood flow in each area of the brain. This signal doesn't directly track neural activity! After there is a burst of neural activity in an area, nearby blood vessels slowly respond and recruit more blood over the next 2-8 seconds. So we can't directly model the fMRI response with our stimuli! We need to account for the slow [hemodynamic response](https://en.wikipedia.org/wiki/Haemodynamic_response).

We can think of the hemodynamic response as (to first approximation) a _filter_ on the underlying neural activity. So if the stimulus features (here, the presence of people or buildings in a video) is correlated with the neural activity, we can try to find a filter that we can apply to the stimulus features in order to make them look like the response.

We'll do this by creating new features that are _lagged_ versions of the stimulus features. If we use lags (1,2,3,4) then it's like we're modeling the response $y_t$ as a function of the stimulus features $x_{t-1}, x_{t-2}, x_{t-3}, x_{t-4}$. This is just like fitting a filter that is (effectively) convolved with the stimulus feature to get the response!

In [ ]:
# let's start with just one feature, feature zero
# create a matrix that will hold 4 lagged feature vectors
X_trn_lagged = np.zeros((###SOMETHING###))

In [ ]:
# now copy the feature X_trn[:,0] into each of the 4 columns of your new matrix
# we want the first column to be lagged by 1 timepoint relative to the original,
# the second is lagged 2, etc.

for ii,lag in enumerate([1,2,3,4]):
    # `frm` should be the timepoint that you start copying from in X_trn
    frm = ## SOMETHING ##
    # and `to` is the last timepoint
    to = ## SOMETHING
    X_trn_lagged[lag:,ii] = X_trn[frm:to,0]

In [ ]:
# plot each of these 4 feature vectors on the same plot here to make sure they look right
# they should look identical, but shifted in time

In [ ]:
# now do the same thing to create X_val_lagged

## Fit the model!
Next, use `np.linalg.lstsq` to fit a linear regression model that predicts `y_trn` from `X_trn_lagged`. Save the `wt, rank, res, sing` as before.

In [19]:
# do regression here

In [20]:
# now plot the weights as dots connected by lines. what do these weights mean? why do they look like that?

## Test the model!
Use the dot product between `wt` and `X_val_lagged` to predict responses in the test dataset, then compute the correlation between them. How well does our model work?

In [ ]:
# create predicted test timecourse
y_test_pred = ## SOMETHING ##

In [ ]:
# compute correlation
model1_corr = ## SOMETHING ##
print(model1_corr)

# Repeat the modeling procedure for every voxel in the `ybig_*` dataset
The model above was fit to just one voxel. Let's repeat this process for each of the 10,000 voxels in the bigger dataset. Ideally we would do this by reshaping the `ybig_*` datasets into a more sensible shape, but it might be easier to do using a for-loop.

In [ ]:
wt_big = np.zeros((4, 100, 100))
for ii in range(100):
    for jj in range(100):
        ## SOMETHING ##
        wt_big[:,ii,jj] = ## SOMETHING ##

In [ ]:
# compute predictions & find the prediction correlation for every voxel in the big set
corr_big = np.zeros((100,100))
for ii in range(100):
    for jj in range(100):
        corr_big[ii,jj] = ## SOMETHING ##

In [ ]:
# plot the correlations using plt.matshow. 
# this shows correlation across one axial slice through a subject's brain. what do you see?